In [3]:
import itertools
from datetime import datetime, time
import pandas as pd

# Data jadwal yang disesuaikan dengan gambar
schedule_data = [
    {
        'no': 1,
        'subject': 'Professional English',
        'room': 'B403 / LabA215',
        'time1': 'Mon, 14:30-16:45',
        'time2': 'Thu, 14:30-16:45'
    },
    {
        'no': 2,
        'subject': 'Data Science Project',
        'room': 'LabA216 / LabA216',
        'time1': 'Mon, 07:00-09:15',
        'time2': 'Tue, 07:00-09:15'
    },
    {
        'no': 3,
        'subject': 'Data Analytic and Visualization Project',
        'room': 'LabA216 / LabA216',
        'time1': 'Wed, 07:00-09:15',
        'time2': 'Thu, 07:00-09:15'
    },
    {
        'no': 4,
        'subject': 'Big Data Project',
        'room': 'LabA216 / LabA216',
        'time1': 'Wed, 09:30-11:45',
        'time2': 'Fri, 07:00-09:15'
    }
]


In [4]:
def parse_time_slot(time_str):
    """Parse time slot string menjadi (day, start_time, end_time)"""
    time_str = time_str.strip()
    
    # Mapping hari
    day_mapping = {
        'MON': 'Monday', 'Mon': 'Monday',
        'TUE': 'Tuesday', 'Tue': 'Tuesday', 
        'WED': 'Wednesday', 'Wed': 'Wednesday',
        'THU': 'Thursday', 'Thu': 'Thursday',
        'FRI': 'Friday', 'Fri': 'Friday'
    }
    
    # Split berdasarkan koma
    parts = time_str.split(',')
    day_part = parts[0].strip()
    time_part = parts[1].strip()
    
    # Get day
    day = day_mapping.get(day_part, day_part)
    
    # Parse time range
    if '-' in time_part:
        start_str, end_str = time_part.split('-')
        start_str = start_str.strip()
        end_str = end_str.strip()
        
        # Convert to time objects
        start_time = datetime.strptime(start_str, '%H:%M').time()
        end_time = datetime.strptime(end_str, '%H:%M').time()
        
        return (day, start_time, end_time)
    
    return None

def check_time_conflict(slot1, slot2):
    """Check if two time slots conflict"""
    if slot1 is None or slot2 is None:
        return False
        
    day1, start1, end1 = slot1
    day2, start2, end2 = slot2
    
    # Jika hari berbeda, tidak ada konflik
    if day1 != day2:
        return False
    
    # Check overlap pada hari yang sama
    # Konflik jika: start1 < end2 and start2 < end1
    return start1 < end2 and start2 < end1

def generate_all_combinations():
    """Generate semua kombinasi jadwal yang mungkin"""
    # Setiap subject memiliki 2 pilihan waktu
    choices = []
    for subject in schedule_data:
        time1 = parse_time_slot(subject['time1'])
        time2 = parse_time_slot(subject['time2'])
        choices.append([
            (subject['subject'], 1, time1),
            (subject['subject'], 2, time2)
        ])
    
    # Generate semua kombinasi
    all_combinations = list(itertools.product(*choices))
    return all_combinations

def check_combination_conflicts(combination):
    """Check if a combination has any conflicts"""
    time_slots = [item[2] for item in combination]
    
    # Check setiap pasangan time slot
    for i in range(len(time_slots)):
        for j in range(i + 1, len(time_slots)):
            if check_time_conflict(time_slots[i], time_slots[j]):
                return True, (combination[i][0], combination[j][0])
    
    return False, None

def format_schedule(combination):
    """Format jadwal combination untuk display"""
    schedule = {}
    for subject, choice, time_slot in combination:
        if time_slot:
            day, start_time, end_time = time_slot
            if day not in schedule:
                schedule[day] = []
            schedule[day].append({
                'subject': subject,
                'time': f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}",
                'choice': choice
            })
    
    # Sort by day and time
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    sorted_schedule = {}
    for day in day_order:
        if day in schedule:
            sorted_schedule[day] = sorted(schedule[day], key=lambda x: x['time'])
    
    return sorted_schedule

# Generate semua kombinasi
print("🔍 Menganalisis semua kemungkinan kombinasi jadwal...")
all_combinations = generate_all_combinations()
print(f"Total kombinasi yang mungkin: {len(all_combinations)}")

# Filter kombinasi yang tidak konflik
valid_combinations = []
conflict_details = []

for i, combination in enumerate(all_combinations):
    has_conflict, conflict_subjects = check_combination_conflicts(combination)
    if not has_conflict:
        valid_combinations.append(combination)
    else:
        conflict_details.append((i+1, conflict_subjects))

print(f"\n✅ Kombinasi jadwal yang VALID (tanpa konflik): {len(valid_combinations)}")
print(f"❌ Kombinasi jadwal yang KONFLIK: {len(conflict_details)}")

# Tampilkan semua jadwal valid
print("\n" + "="*80)
print("JADWAL VALID YANG TIDAK BERTABRAKAN:")
print("="*80)

for idx, combination in enumerate(valid_combinations, 1):
    print(f"\n📅 OPSI JADWAL {idx}:")
    print("-" * 50)
    
    # Show choices for each subject
    for subject, choice, time_slot in combination:
        if time_slot:
            day, start_time, end_time = time_slot
            print(f"• {subject}: Pilihan {choice}")
            print(f"  └─ {day}, {start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}")
    
    # Show formatted weekly schedule
    print(f"\n📊 JADWAL MINGGUAN OPSI {idx}:")
    formatted_schedule = format_schedule(combination)
    
    for day, classes in formatted_schedule.items():
        print(f"\n{day}:")
        for class_info in classes:
            print(f"  • {class_info['time']} - {class_info['subject']}")

print("\n" + "="*80)
print("RINGKASAN:")
print("="*80)
print(f"✅ Jumlah opsi jadwal valid: {len(valid_combinations)}")
print("✅ Semua opsi di atas dijamin tidak ada bentrok waktu")
print("✅ Anda bisa memilih salah satu dari opsi jadwal tersebut")

if len(valid_combinations) == 0:
    print("\n⚠️  PERINGATAN: Tidak ada kombinasi jadwal yang valid!")
    print("Ada konflik waktu yang tidak dapat dihindari dengan pilihan yang tersedia.")
    
    print(f"\n❌ Detail konflik pada {len(conflict_details)} kombinasi:")
    for combo_num, conflict_subjects in conflict_details[:5]:  # Show first 5 conflicts
        print(f"  • Kombinasi {combo_num}: Konflik antara '{conflict_subjects[0]}' dan '{conflict_subjects[1]}'")

🔍 Menganalisis semua kemungkinan kombinasi jadwal...
Total kombinasi yang mungkin: 16

✅ Kombinasi jadwal yang VALID (tanpa konflik): 16
❌ Kombinasi jadwal yang KONFLIK: 0

JADWAL VALID YANG TIDAK BERTABRAKAN:

📅 OPSI JADWAL 1:
--------------------------------------------------
• Professional English: Pilihan 1
  └─ Monday, 14:30 - 16:45
• Data Science Project: Pilihan 1
  └─ Monday, 07:00 - 09:15
• Data Analytic and Visualization Project: Pilihan 1
  └─ Wednesday, 07:00 - 09:15
• Big Data Project: Pilihan 1
  └─ Wednesday, 09:30 - 11:45

📊 JADWAL MINGGUAN OPSI 1:

Monday:
  • 07:00 - 09:15 - Data Science Project
  • 14:30 - 16:45 - Professional English

Wednesday:
  • 07:00 - 09:15 - Data Analytic and Visualization Project
  • 09:30 - 11:45 - Big Data Project

📅 OPSI JADWAL 2:
--------------------------------------------------
• Professional English: Pilihan 1
  └─ Monday, 14:30 - 16:45
• Data Science Project: Pilihan 1
  └─ Monday, 07:00 - 09:15
• Data Analytic and Visualization Proj